In [1]:
from src.lexer.hulk_lexer import HulkLexer
from src.hulk_grammar.hulk_grammar import G
from src.parsing import LR1Parser


lexer = HulkLexer()
parser = LR1Parser(G)

In [5]:
from src.evaluation import evaluate_reverse_parse
from src.semantics.semantic_analysis_pipeline import semantic_analysis_pipeline


tokens, errors = lexer('''
    
    let a : Number = 3 as Object, b : Number = 5 in print(a);
    '''
)

derivation, operations = parser([t.token_type for t in tokens])
ast = evaluate_reverse_parse(derivation, operations, tokens)
ast, errors, context, scope = semantic_analysis_pipeline(ast, debug=True)

===================== AST =====================
\__ ProgramNode [<stat>; ... <stat> <expr>;]

	\__ LetInNode: let [<var>, ..., <var>] in <expr>
		\__ VarDeclarationNode: a = <expr>
			\__ AsNode: <expr> as Object
				\__ ConstantNumNode: 3
		\__ VarDeclarationNode: b = <expr>
			\__ ConstantNumNode: 5
		\__ CallNode: print(<expr>, ..., <expr>)
			\__ VariableNode: a
============== COLLECTING TYPES ===============
Errors: [
]
Context:
{
	type Object {}
	
	type String inherits Object {}
	
	type Number inherits Object {}
	
	type Bool inherits Object {}
	
	function print(value:Object): String;
	
	
	function sqrt(value:Number): Number;
	
	
	function sin(angle:Number): Number;
	
	
	function cos(angle:Number): Number;
	
	
	function exp(value:Number): Number;
	
	
	function log(base:Number, value:Number): Number;
	
	
	function rand(): Number;
	
}
=============== BUILDING TYPES ================
Errors: [
]
Context:
{
	type Object {}
	
	type String inherits Object {}
	
	type Number inherits Objec

In [7]:
from src.code_gen.expression_visitor import CodeGenC

codgen = CodeGenC(context)

code = ""

create_defs = {}
method_defs = {}

for type in context.types.values():
    if type.name not in ["Number", "Bool", "String", "Object"]:
        create_def = "Object* create" + type.name + " ("
        create_params = []

        current = type

        while current is not None:
            for att in current.attributes:
                create_params.append(current.name + "_" + att.name)
                create_def += "Object* " + create_params[-1] + ", "

            current = current.parent

        if len(create_params) > 0:
            create_def = create_def[:-2]
        
        create_def += ")"

        create_defs[type.name] = (create_def, create_params)

        code += create_def + ";\n"

        method_defs[type.name] = []

        for method in type.methods:
            method_name = "fun_" + type.name + "_" + method.name
            method_def = "Object* " + method_name + " (Object* self"
            
            method_id_params = {}
            
            for param_name in method.param_names:
                id_param = "p_" + param_name
                method_id_params[param_name] = id_param
                method_def += ", Object* " + id_param

            method_def += ")"
            method_defs[type.name].append((method_def, method_name, method_id_params, method))
            code += method_def + ";\n"
                
        code += "\n"

for type in context.types.values():
    if type.name not in ["Number", "Bool", "String", "Object"]:
        code += create_defs[type.name][0] + " {\n"
        code += "   Object* obj = createObject();\n"
        code += "   addAttribute(obj, \"type\",\"" + type.name + "\");\n"

        code += "\n"
        for param in create_defs[type.name][1]:
            code += "   addAttribute(obj, \"" + param + "\", " + param + ");\n"

        code += "\n"

        current = type

        while current is not None and current.name != "Object":
            if current.name in method_defs:
                for method in method_defs[current.name]:
                    code += "   addAttribute(obj, \"" + method[1] + "\", *" + method[1] + ");\n"

            current = current.parent

        code += "}\n\n"

for type in context.types.values():
    if type.name not in ["Number", "Bool", "String", "Object"]:
        if type.name in method_defs:
            for method_def, method_name, method_id_params, method in method_defs[type.name]:
                code += method_def + " {\n"

                code += codgen.visit(method.node)[0] + "\n"

                code += "}\n\n"
        
for function in context.functions.values():
    if function.name not in ['print', 'sqrt', 'sin', 'cos', 'exp', 'log', 'rand']:
        code += codgen.visit(function.node)[0] + "\n"

code += "int main() {\n"

code += codgen.visit(ast.expression)[0] + "\n"

code += "return 0; \n}"

print(code)

IndexError: list index out of range